In [105]:
from mistletoe.span_token import SpanToken
import re

class PersonalInfo(SpanToken):
    pattern = re.compile(r'`(.+?)`.*?\[(.+?)\].*\|?')
    precedence = 6
    def __init__(self, match_obj):
        self.target = match_obj.group(2)

class BeginDocument(SpanToken):
    pattern = re.compile(r' {0,3}(?:([_])\s*?)(?:\1\s*?){2,}')
    def __init__(self, match_obj):
        self.target = match_obj.group(1)

class PageBreak(SpanToken):
    pattern = re.compile(r' {0,3}(?:([*])\s*?)(?:\1\s*?){2,}')
    precedence = 6
    def __init__(self, match_obj):
        pass

In [106]:
from mistletoe.block_token import BlockToken, Heading, CodeFence, ThematicBreak, List, Paragraph, tokenizer, _token_types
import re

ThematicBreak.pattern = re.compile(r' {0,3}(?:([-])\s*?)(?:\1\s*?){2,}$')

class CvEntry(BlockToken):
    """
    Block quote token. (["> # heading\\n", "> paragraph\\n"])
    This is a container block token. Its children are block tokens - container or leaf ones.
    """
    def __init__(self, parse_buffer):
        # span-level tokenizing happens here.
        self.children = tokenizer.make_tokens(parse_buffer)

    @staticmethod
    def start(line):
        stripped = line.lstrip(' ')
        if len(line) - len(stripped) > 3:
            return False
        return stripped.startswith('###')

    @classmethod
    def read(cls, lines):
        # first line
        line = cls.convert_leading_tabs(next(lines).lstrip()).split('###', 1)[1]
        if len(line) > 0 and line[0] == ' ':
            line = line[1:]
        line_buffer = [line]

        # loop
        next_line = lines.peek()
        if (next_line is not None
                and not Heading.start(next_line)
                and not CodeFence.start(next_line)
                and not ThematicBreak.start(next_line)
                and not List.start(next_line)):

            # probably a nicer way to avoid this being counted as a cvitem
            stripped = '<BLANK>'
            if next_line.strip() != '':
                stripped = cls.convert_leading_tabs(next_line.lstrip())
            line_buffer.append(stripped)
            next(lines)
        
        # block level tokens are parsed here, so that footnotes
        # in quotes can be recognized before span-level tokenizing.
        Paragraph.parse_setext = False
        parse_buffer = tokenizer.tokenize_block(line_buffer, _token_types)
        Paragraph.parse_setext = True
        return parse_buffer

    @staticmethod
    def convert_leading_tabs(string):
        string = string.replace('>\t', '   ', 1)
        count = 0
        for i, c in enumerate(string):
            if c == '\t':
                count += 4
            elif c == ' ':
                count += 1
            else:
                break
        if i == 0:
            return string
        return '>' + ' ' * count + string[i:]
    

In [107]:
from mistletoe.latex_renderer import LaTeXRenderer
from mistletoe.span_token import RawText

SOCIALS = ['github', 'gitlab', 'twitter', 'linkedin']
PHONES = ['mobile', 'fixed', 'fax']

class ModernCVRenderer(LaTeXRenderer):
    def __init__(self):
        super().__init__(CvEntry, PersonalInfo, BeginDocument, PageBreak)
        self.packages['babel'] = '[english]'
        self.packages['eurosym'] = []
        self.packages['geometry'] = '[scale=.85]'
        self.packages['lmodern'] = []
    
    def render_document(self, token):
        template = ('\\documentclass[10pt,a4paper,sans]{{moderncv}}\n'
                    '\\moderncvstyle{{classic}}\n'
                    '\\moderncvcolor{{blue}}\n'
                    '\\renewcommand{{\\listitemsymbol}}{{-~}}\n'
                    '\\nopagenumbers{{}}\n'
                    '{packages}'
                    '\\setlength{{\\hintscolumnwidth}}{{3cm}}\n\n'
                    '{inner}'
                    '\\end{{document}}\n')
        self.footnotes.update(token.footnotes)
        return template.format(inner=self.render_inner(token),
                               packages=self.render_packages())

    def render_heading(self, token):
        inner = self.render_inner(token)
        if token.level == 1:
            template = ('\\name{{{}}}{{{}}}\n'
                    '\\title{{{}}}')
            split_title = inner.split('-')
            name = split_title[0].strip().split(' ')
            return template.format(*name, split_title[1].strip())
        elif token.level == 2:
            return '\n\\section{{{}}}\n'.format(inner)
        elif token.level == 3:
            return '\n\\subsection{{{}}}\n'.format(inner)
        return '\n\\subsubsection{{{}}}\n'.format(inner)

    def render_personal_info(self, token):
        template = '\{inner}{{{target}}}'
        target = token.target
        inner = self.render_inner(token)
        if inner in SOCIALS:
            inner = 'social[{inner}]'.format(inner=inner)
        if inner in PHONES:
            inner = 'phone[{inner}]'.format(inner=inner)
        return template.format(target=target, inner=inner)

    def render_begin_document(self, token):
        template = ('\\begin{document}\n'
                    '\\makecvtitle')
        return template

    def render_page_break(self, token):
        return '\\clearpage\n'

    def render_paragraph(self, token):
        if all(isinstance(item, RawText) for item in token.children):
            return '\n\\cvitem{{}}{{{}}}\n'.format(self.render_inner(token))
        return super().render_paragraph(token)

    def render_list(self, token):
        return self.render_inner(token)

    def render_list_item(self, token):
        inner = ''
        if all(isinstance(item, RawText) for item in token.children[0].children):
            inner = ' '.join([self.render_raw_text(child) for child in token.children[0].children])
        return '\\cvlistitem{{{}}}\n'.format(inner)

    def convert_pipe_to_bracket(string):
        return string.replace('|', '}{')

    def render_cv_entry(self, token):
        # [cls.convert_pipe_to_bracket(line) + '}' + '{{}}' * (4 - line.count('|')) + '{']
        
        lines = list(filter(None, self.render_inner(token).split('\n')))
        if len(lines) != 2:
            raise ValueError
        split_pipe = lines[0].split('|')
        inner_first_section = '}{'.join([x.strip() for x in split_pipe])
        # Should be missing brackets for a cventry if > 1 pipe symbol or missing for cvitem if <= 1
        should_be_item = len(split_pipe) <= 2 and lines[1] == '<BLANK>'
        missing_brackets = 2 - len(split_pipe) if should_be_item else 5 - len(split_pipe)

        first_section_template = '\\cvitem{{{}}}' if should_be_item else '\\cventry{{{}}}'
        first_section = first_section_template.format(inner_first_section)
        ending = '' if should_be_item else '{{{}}}'.format(lines[1].replace('<BLANK>', ''))
        return first_section + '{}' * missing_brackets + ending + '\n'

    def render_table(self, token):
        def render_align(column_align):
            if column_align != [None]:
                cols = [get_align(col) for col in token.column_align]
                return '{{{}}}'.format(' '.join(cols))
            else:
                return ''

        def get_align(col):
            if col is None:
                return 'l'
            elif col == 0:
                return 'p{2cm}'
            elif col == 1:
                return '|r'
            raise RuntimeError('Unrecognized align option: ' + col)

        template = ('\\setlength{{\\tabcolsep}}{{0.5em}}\n'
                    '{{\\renewcommand{{\\arraystretch}}{{1.2}}\n'
                    '\\begin{{tabular}}{align}\n'
                    '{inner}'
                    '\\end{{tabular}}}}\n')
        if hasattr(token, 'header'):
            head_template = '{inner}\\hline\n'
            head_inner = self.render_table_row(token.header)
            head_rendered = head_template.format(inner=head_inner)
        else: 
            head_rendered = ''
        inner = self.render_inner(token)
        align = render_align(token.column_align)
        return template.format(inner=head_rendered+inner, align=align)

In [108]:
from mistletoe import Document

rendered = None
with open('cv.md', 'r') as fin:
    with ModernCVRenderer() as renderer:
        rendered = renderer.render(Document(fin))

In [109]:
with open('cv.tex', 'w') as fout:
    fout.write(rendered)

In [110]:
import html
from mistletoe.html_renderer import HTMLRenderer
from mistletoe.span_token import RawText
from mistletoe.block_token import ThematicBreak

ThematicBreak.pattern = re.compile(r' {0,3}(?:([-])\s*?)(?:\1\s*?){2,}$')

class MudBlazorCVRenderer(HTMLRenderer):
    def __init__(self):
        super().__init__(PersonalInfo, BeginDocument, PageBreak)
        self.first = True

    def render_document(self, token):
        template = ('<MudList Clickable="false">\n'
                    '{inner}'
                    '</NestedList>\n'
                    '</MudListItem>\n'
                    '</MudList>')
        self.footnotes.update(token.footnotes)
        return template.format(inner=self.render_inner(token))

    def render_personal_info(self, token):
        return ''

    def render_paragraph(self, token):
        inner_render = self.render_inner(token)
        if self._suppress_ptag_stack[-1]:
            return '{}'.format(inner_render)

        if inner_render is None or inner_render == '' or inner_render.isspace():
            return ''
        if all(isinstance(item, RawText) for item in token.children):
            return '<MudListItem>\n{}</MudListItem>\n'.format(inner_render)
        else:
            return '<MudListItem>\n<NestedList>\n{}</NestedList>\n</MudListItem>\n'.format(inner_render)
    
    def render_raw_text(self, token):
        return '{}\n'.format(html.escape(token.content))
    
    def render_begin_document(self, token):
        return ''

    def render_page_break(self, token):
        return ''
    
    def render_list(self, token):
        template = '<MudListItem>\n<NestedList>\n{inner}</NestedList>\n</MudListItem>\n'
        self._suppress_ptag_stack.append(not token.loose)
        inner = '\n'.join([self.render(child) for child in token.children])
        self._suppress_ptag_stack.pop()
        return template.format(inner=inner)
    
    def render_list_item(self, token):
        if len(token.children) == 0:
            return '    <MudListItem></MudListItem>\n'
        inner = '\n'.join([self.render(child) for child in token.children])
        inner_template = '\n{}\n'
        if self._suppress_ptag_stack[-1]:
            if token.children[0].__class__.__name__ == 'Paragraph':
                inner_template = inner_template[1:]
            if token.children[-1].__class__.__name__ == 'Paragraph':
                inner_template = inner_template[:-1]
        return '    <MudListItem>\n{}</MudListItem>\n'.format(inner_template.format(inner))
    
    def render_heading(self, token):
        inner = self.render_inner(token)
        if token.level == 1:
            return ''
        elif token.level == 2:
            if self.first:
                self.first = False
                return '<MudListItem>\n<ChildContent>\n<MudText Class="mud-typography-h4">{}\n</MudText>\n</ChildContent>\n<NestedList>\n'.format(super().render_heading(token))
            else:
                return '</NestedList>\n</MudListItem>\n<MudListItem><ChildContent>\n<MudText Class="mud-typography-h4">{}\n</MudText>\n</ChildContent><NestedList>\n'.format(super().render_heading(token))

        return '<MudListItem>\n{}</MudListItem>\n'.format(super().render_heading(token))

    def render_table(self, token):
        # This is actually gross and I wonder if there's a better way to do it.
        #
        # The primary difficulty seems to be passing down alignment options to
        # reach individual cells.
        template = '<MudSimpleTable>\n{inner}</MudSimpleTable>'
        if hasattr(token, 'header'):
            head_template = '<thead>\n{inner}</thead>\n'
            head_inner = self.render_table_row(token.header, is_header=True)
            head_rendered = head_template.format(inner=head_inner)
        else: head_rendered = ''
        body_template = '<tbody>\n{inner}</tbody>\n'
        body_inner = self.render_inner(token)
        body_rendered = body_template.format(inner=body_inner)
        return template.format(inner=head_rendered+body_rendered)

In [111]:
from mistletoe import Document

rendered_html = None
with open('cv.md', 'r') as fin:
    with MudBlazorCVRenderer() as renderer:
        rendered_html = renderer.render(Document(fin))

In [112]:
with open('cv.razor', 'w') as fout:
    fout.write(rendered_html)